In [1]:
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
def calculate_psnr(img1, img2):
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr



/home/nbicc/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
path1 = 'results/elic_vtest/label/0001.png'
path2 = 'results/gg18_zoo_hypergg18/recon/test0000.png'
# path2 = 'results/elic_vtest/input/test0000.png'
# path2 = 'results/elic_vtest/recon/gg18_test/x0_sub_A(measure).png'
# path2 = 'data/test3/00000.png'
# 示例用法
# 读取两张图像
img1 = Image.open(path2)
img2 = Image.open(path1)
img1 = img1.convert("RGB")
img2 = img2.convert("RGB")

# 将图像转换为 NumPy 数组
img1 = np.array(img1)
img2 = np.array(img2)

# 确保图像具有相同的尺寸和类型
if img1.shape != img2.shape:
    raise ValueError("输入图像必须具有相同的尺寸和类型")

psnr_value = calculate_psnr(img1, img2)
print(f"PSNR: {psnr_value} dB")


PSNR: 34.98064812322487 dB


In [9]:
import torch
x = torch.tensor(-1.1839e-2)
print(torch.round(x))

tensor(-0.)


In [2]:

# library
# standard library
import os

# third-party library
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
# import matplotlib.pyplot as plt

# torch.manual_seed(1)    # reproducible

# Hyper Parameters
EPOCH = 1               # train the training data n times, to save time, we just train 1 epoch
BATCH_SIZE = 50
LR = 0.001              # learning rate
DOWNLOAD_MNIST = False


# Mnist digits dataset
if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True

train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,                                     # this is training data
    transform=torchvision.transforms.ToTensor(),    # Converts a PIL.Image or numpy.ndarray to
                                                    # torch.FloatTensor of shape (C x H x W) and normalize in the range [0.0, 1.0]
    download=DOWNLOAD_MNIST,
)

# plot one example
print(train_data.train_data.size())                 # (60000, 28, 28)
print(train_data.train_labels.size())               # (60000)
# plt.imshow(train_data.train_data[0].numpy(), cmap='gray')
# plt.title('%i' % train_data.train_labels[0])
# plt.show()

# Data Loader for easy mini-batch return in training, the image batch shape will be (50, 1, 28, 28)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# pick 2000 samples to speed up testing
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255.   # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.test_labels[:2000]


class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(28*28,28*28),
            nn.Linear(28*28,10)
        )

    def forward(self, x):
        output = self.mlp(x)
        return output, x    # return x for visualization


mlp = MLP()
print(mlp)  # net architecture

optimizer = torch.optim.Adam(mlp.parameters(), lr=LR)   # optimize all logistic parameters
loss_func = nn.CrossEntropyLoss()                       # the target label is not one-hotted

# following function (plot_with_labels) is for visualization, can be ignored if not interested
# from matplotlib import cm
# try: from sklearn.manifold import TSNE; HAS_SK = True
# except: HAS_SK = False; print('Please install sklearn for layer visualization')
# def plot_with_labels(lowDWeights, labels):
#     plt.cla()
#     X, Y = lowDWeights[:, 0], lowDWeights[:, 1]
#     for x, y, s in zip(X, Y, labels):
#         c = cm.rainbow(int(255 * s / 9)); plt.text(x, y, s, backgroundcolor=c, fontsize=9)
#     plt.xlim(X.min(), X.max()); plt.ylim(Y.min(), Y.max()); plt.title('Visualize last layer'); plt.show(); plt.pause(0.01)

# plt.ion()
# training and testing
for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):   # gives batch data, normalize x when iterate train_loader
        # print(b_x.size())
        b_x = b_x.view(-1, 28*28)
        # print(b_x.size())

        output = mlp(b_x)[0]               # logistic output
        loss = loss_func(output, b_y)   # cross entropy loss
        optimizer.zero_grad()           # clear gradients for this training step
        loss.backward()                 # backpropagation, compute gradients
        optimizer.step()                # apply gradients

        if step % 50 == 0:

            test_output, last_layer = mlp(test_x.view(-1,28*28))
            pred_y = torch.max(test_output, 1)[1].data.numpy()
            accuracy = float((pred_y == test_y.data.numpy()).astype(int).sum()) / float(test_y.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| test accuracy: %.2f' % accuracy)
            # if HAS_SK:
            #     # Visualization of trained flatten layer (T-SNE)
            #     tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
            #     plot_only = 500
            #     low_dim_embs = tsne.fit_transform(last_layer.data.numpy()[:plot_only, :])
            #     labels = test_y.numpy()[:plot_only]
            #     plot_with_labels(low_dim_embs, labels)
# plt.ioff()


/home/nbicc/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/nbicc/data/anaconda3/envs/Idem_project/lib/python3.9/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/home/nbicc/data/anaconda3/envs/Idem_project/lib/python3.9/site-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/home/nbicc/data/anaconda3/envs/Idem_project/lib/python3.9/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/nbicc/data/anaconda3/envs/Idem_project/lib/python3.9/site-packages/torchvision/datasets/mnist.py:70: UserWarning: test_

torch.Size([60000, 28, 28])
torch.Size([60000])
MLP(
  (mlp): Sequential(
    (0): Linear(in_features=784, out_features=784, bias=True)
    (1): Linear(in_features=784, out_features=10, bias=True)
  )
)
Epoch:  0 | train loss: 2.3211 | test accuracy: 0.35
Epoch:  0 | train loss: 0.5372 | test accuracy: 0.84
Epoch:  0 | train loss: 0.5481 | test accuracy: 0.87
Epoch:  0 | train loss: 0.4311 | test accuracy: 0.87
Epoch:  0 | train loss: 0.4135 | test accuracy: 0.88
Epoch:  0 | train loss: 0.4153 | test accuracy: 0.87
Epoch:  0 | train loss: 0.3869 | test accuracy: 0.89
Epoch:  0 | train loss: 0.3906 | test accuracy: 0.89
Epoch:  0 | train loss: 0.2951 | test accuracy: 0.88
Epoch:  0 | train loss: 0.4829 | test accuracy: 0.89
Epoch:  0 | train loss: 0.1639 | test accuracy: 0.89
Epoch:  0 | train loss: 0.2176 | test accuracy: 0.88
Epoch:  0 | train loss: 0.1714 | test accuracy: 0.88
Epoch:  0 | train loss: 0.2197 | test accuracy: 0.89
Epoch:  0 | train loss: 0.5813 | test accuracy: 0.88
Ep

In [3]:
print(test_x.shape)

torch.Size([2000, 1, 28, 28])


In [31]:
tmp = test_x[0].view(-1,28*28).requires_grad_()
test_output, _ = mlp(tmp)



In [29]:
jac = torch.autograd.functional.jacobian(mlp, tmp)
jacobian_matrix = jac[0].squeeze()
print(jac[0].squeeze().shape)
print(test_output.shape)

torch.Size([10, 784])
torch.Size([1, 10])


In [32]:
norm = torch.linalg.norm(test_output)
norm_grad = torch.autograd.grad(outputs=norm, inputs=tmp)


In [40]:
print(type(norm_grad[0]))

<class 'torch.Tensor'>


In [43]:
print(torch.mm(test_output, jacobian_matrix) / norm)

tensor([[-6.0759e-03, -3.7439e-02, -1.3934e-02, -4.6441e-02,  3.1168e-02,
         -1.6971e-02, -4.2810e-02,  1.3516e-02, -6.6781e-02, -4.5483e-02,
         -6.0959e-03, -1.2523e-02, -2.9472e-01, -3.1187e-01,  8.6154e-02,
         -2.5666e-02, -2.6967e-02, -1.2780e-02, -7.1050e-03, -2.6533e-02,
         -6.9585e-02, -5.3721e-03, -1.6798e-02, -1.3750e-02,  1.9407e-02,
          3.9355e-02, -2.7501e-02,  7.6976e-03, -1.1780e-02,  2.6902e-02,
         -9.0492e-03,  3.2159e-02, -9.5543e-02, -2.4021e-01, -5.9225e-01,
         -5.1630e-01, -3.0749e-01, -5.9233e-01, -6.6773e-01, -5.6813e-01,
         -8.1137e-01, -6.6458e-01, -4.4653e-01, -2.2380e-01,  1.3606e-01,
         -1.5265e-01, -8.3959e-01, -1.0065e+00, -1.0586e+00, -7.1316e-01,
         -4.1854e-01, -2.3793e-01, -5.4846e-03, -1.2962e-02,  2.6680e-02,
         -4.8496e-02,  3.6268e-03, -2.0028e-02, -1.2501e-01,  1.9347e-02,
         -4.4791e-01, -6.7392e-01, -5.4180e-01, -6.2856e-01, -6.0256e-01,
         -5.9289e-01, -8.3145e-01, -6.

In [44]:
print(norm_grad[0])

tensor([[-6.0759e-03, -3.7439e-02, -1.3934e-02, -4.6441e-02,  3.1168e-02,
         -1.6971e-02, -4.2810e-02,  1.3516e-02, -6.6781e-02, -4.5483e-02,
         -6.0959e-03, -1.2523e-02, -2.9472e-01, -3.1187e-01,  8.6154e-02,
         -2.5666e-02, -2.6967e-02, -1.2780e-02, -7.1050e-03, -2.6533e-02,
         -6.9585e-02, -5.3721e-03, -1.6798e-02, -1.3750e-02,  1.9407e-02,
          3.9355e-02, -2.7501e-02,  7.6976e-03, -1.1780e-02,  2.6902e-02,
         -9.0492e-03,  3.2159e-02, -9.5543e-02, -2.4021e-01, -5.9225e-01,
         -5.1630e-01, -3.0749e-01, -5.9233e-01, -6.6773e-01, -5.6813e-01,
         -8.1137e-01, -6.6458e-01, -4.4653e-01, -2.2380e-01,  1.3606e-01,
         -1.5265e-01, -8.3959e-01, -1.0065e+00, -1.0586e+00, -7.1316e-01,
         -4.1854e-01, -2.3793e-01, -5.4846e-03, -1.2962e-02,  2.6680e-02,
         -4.8496e-02,  3.6267e-03, -2.0028e-02, -1.2501e-01,  1.9347e-02,
         -4.4791e-01, -6.7392e-01, -5.4180e-01, -6.2856e-01, -6.0256e-01,
         -5.9289e-01, -8.3145e-01, -6.

In [45]:
tmp = test_x[1].view(-1,28*28).requires_grad_()
test_output, _ = mlp(tmp)
norm = torch.linalg.norm(test_output)
norm_grad = torch.autograd.grad(outputs=norm, inputs=tmp)

In [46]:
print(norm_grad[0])

tensor([[ 2.7136e-02,  6.0319e-02, -7.6147e-03,  4.4035e-02, -3.1631e-03,
         -2.7548e-02,  6.3854e-02, -1.7567e-02,  5.8918e-02,  1.8257e-02,
          9.2602e-03, -3.4233e-03,  4.5773e-01,  5.4676e-01,  1.5570e-02,
         -2.7559e-04, -6.6829e-03, -6.2579e-03, -3.5207e-02, -1.4308e-02,
          3.5341e-02,  6.8840e-03,  1.3651e-02,  1.1460e-02, -4.5680e-04,
         -2.3589e-02,  3.1409e-02, -1.4937e-03, -7.7611e-03, -3.9546e-02,
          4.8275e-03, -3.8038e-02,  2.8741e-01,  4.1074e-01,  8.6795e-01,
          7.4969e-01,  3.1220e-01,  7.1558e-01,  9.3999e-01,  1.1586e+00,
          1.3185e+00,  8.7183e-01,  3.0443e-01,  7.0659e-01,  1.0205e+00,
          8.7802e-01,  1.2107e+00,  1.1095e+00,  1.2435e+00,  9.6806e-01,
          7.6482e-01,  4.3077e-01,  1.6327e-02,  1.7091e-02, -2.7603e-02,
          3.6041e-02, -2.8854e-02,  1.4958e-02,  2.4431e-01,  4.5503e-01,
          8.6247e-01,  3.4727e-01,  5.0646e-01,  1.1630e+00,  1.0464e+00,
          1.1755e+00,  1.4686e+00,  1.

In [47]:
print(torch.mm(test_output, jacobian_matrix) / norm)

tensor([[ 2.7136e-02,  6.0319e-02, -7.6147e-03,  4.4035e-02, -3.1631e-03,
         -2.7548e-02,  6.3854e-02, -1.7567e-02,  5.8918e-02,  1.8257e-02,
          9.2602e-03, -3.4233e-03,  4.5773e-01,  5.4676e-01,  1.5570e-02,
         -2.7559e-04, -6.6829e-03, -6.2579e-03, -3.5207e-02, -1.4308e-02,
          3.5341e-02,  6.8840e-03,  1.3651e-02,  1.1460e-02, -4.5681e-04,
         -2.3589e-02,  3.1409e-02, -1.4938e-03, -7.7611e-03, -3.9546e-02,
          4.8275e-03, -3.8038e-02,  2.8741e-01,  4.1074e-01,  8.6795e-01,
          7.4969e-01,  3.1220e-01,  7.1558e-01,  9.3999e-01,  1.1586e+00,
          1.3185e+00,  8.7183e-01,  3.0443e-01,  7.0659e-01,  1.0205e+00,
          8.7802e-01,  1.2107e+00,  1.1095e+00,  1.2435e+00,  9.6807e-01,
          7.6482e-01,  4.3077e-01,  1.6327e-02,  1.7091e-02, -2.7603e-02,
          3.6041e-02, -2.8854e-02,  1.4958e-02,  2.4431e-01,  4.5503e-01,
          8.6247e-01,  3.4727e-01,  5.0646e-01,  1.1630e+00,  1.0464e+00,
          1.1755e+00,  1.4686e+00,  1.